In [1]:
import json
import os
import re

# get connection

In [2]:
import get_connections

In [3]:
connection_map = get_connections.main(domain='https://docker.looker.com:19999', url='/api/3.1/connections')

In [4]:
connection_map

{'hub': {'database': 'accounts', 'type': 'PostgreSQL'},
 'registry-hub': {'database': 'repos', 'type': 'PostgreSQL'},
 'data_warehouse': {'database': 'production', 'type': 'Amazon Redshift'},
 'docker_cloud': {'database': 'tutum', 'type': 'MySQL'},
 'nautilus_db': {'database': 'nautilus', 'type': 'PostgreSQL'},
 'restricted': {'database': 'restricted', 'type': 'Amazon Redshift'},
 'data_warehouse_staging': {'database': 'staging', 'type': 'Amazon Redshift'},
 'snowflake_segment': {'database': 'SEGMENT', 'type': 'Snowflake'},
 'snowflake_production': {'database': 'PRODUCTION', 'type': 'Snowflake'},
 'bigquery': {'database': 'clari_pulse_breakdown',
  'type': 'Google BigQuery Standard SQL'},
 'bigquery_production': {'database': 'finance',
  'type': 'Google BigQuery Standard SQL'},
 'snowflake_medium': {'database': 'PRODUCTION', 'type': 'Snowflake'},
 'initiatives-db': {'database': 'initiatives', 'type': 'PostgreSQL'},
 'bigquery_production_marketing': {'database': 'marketing',
  'type': '

# get explore

In [5]:
with open('../../maps/explore-sample_model-sf__accounts.json', 'r') as f:
    explore = json.load(f)

In [6]:
explore

{'explore_name': 'sf__accounts',
 'explore_joins': ['sf__accounts',
  'sf__users',
  'sf__contacts',
  'sf__cases',
  'sf__users'],
 'conn': 'data_warehouse'}

# get views

In [7]:
d = dict()

In [8]:
view_dict = dict()

In [9]:
for view in os.listdir('../../maps'):
    if view.startswith('view'):
        view_dict[view.split('-')[2].split('.')[0]] = view
        with open(f'../../maps/{view}','r') as f:
            payload = json.load(f)
            d[payload['view_name']] = payload

In [65]:
[k,v] = zip(*d.items())

In [70]:
explore

{'explore_name': 'sf__accounts',
 'explore_joins': ['sf__accounts',
  'sf__users',
  'sf__contacts',
  'sf__cases',
  'sf__users'],
 'conn': 'data_warehouse'}

In [71]:
connection_map

{'hub': {'database': 'accounts', 'type': 'PostgreSQL'},
 'registry-hub': {'database': 'repos', 'type': 'PostgreSQL'},
 'data_warehouse': {'database': 'production', 'type': 'Amazon Redshift'},
 'docker_cloud': {'database': 'tutum', 'type': 'MySQL'},
 'nautilus_db': {'database': 'nautilus', 'type': 'PostgreSQL'},
 'restricted': {'database': 'restricted', 'type': 'Amazon Redshift'},
 'data_warehouse_staging': {'database': 'staging', 'type': 'Amazon Redshift'},
 'snowflake_segment': {'database': 'SEGMENT', 'type': 'Snowflake'},
 'snowflake_production': {'database': 'PRODUCTION', 'type': 'Snowflake'},
 'bigquery': {'database': 'clari_pulse_breakdown',
  'type': 'Google BigQuery Standard SQL'},
 'bigquery_production': {'database': 'finance',
  'type': 'Google BigQuery Standard SQL'},
 'snowflake_medium': {'database': 'PRODUCTION', 'type': 'Snowflake'},
 'initiatives-db': {'database': 'initiatives', 'type': 'PostgreSQL'},
 'bigquery_production_marketing': {'database': 'marketing',
  'type': '

In [73]:
args = []
for view in v:
    args.append((view, explore, connection_map))

In [74]:
args

[({'view_name': 'sfbase__accounts',
   'source_table': 'salesforce.accounts',
   'view_type': 'sql_table_name'},
  {'explore_name': 'sf__accounts',
   'explore_joins': ['sf__accounts',
    'sf__users',
    'sf__contacts',
    'sf__cases',
    'sf__users'],
   'conn': 'data_warehouse'},
  {'hub': {'database': 'accounts', 'type': 'PostgreSQL'},
   'registry-hub': {'database': 'repos', 'type': 'PostgreSQL'},
   'data_warehouse': {'database': 'production', 'type': 'Amazon Redshift'},
   'docker_cloud': {'database': 'tutum', 'type': 'MySQL'},
   'nautilus_db': {'database': 'nautilus', 'type': 'PostgreSQL'},
   'restricted': {'database': 'restricted', 'type': 'Amazon Redshift'},
   'data_warehouse_staging': {'database': 'staging',
    'type': 'Amazon Redshift'},
   'snowflake_segment': {'database': 'SEGMENT', 'type': 'Snowflake'},
   'snowflake_production': {'database': 'PRODUCTION', 'type': 'Snowflake'},
   'bigquery': {'database': 'clari_pulse_breakdown',
    'type': 'Google BigQuery Stand

## type 1:  if 'view_type' == 'sql_table_name' 
- warehouse.database.table

In [27]:
d['snowflake_sf__accounts']

{'view_name': 'snowflake_sf__accounts',
 'source_table': 'SEGMENT.SALESFORCE.ACCOUNTS',
 'view_type': 'sql_table_name'}

In [28]:
view_type = d['snowflake_sf__accounts']['view_type']

In [29]:
source1 = d['snowflake_sf__accounts']['source_table']

In [30]:
source1

'SEGMENT.SALESFORCE.ACCOUNTS'

## type 2: if 'view_type' == 'sql_table_name'
- database.table

In [23]:
d['sfbase__cases']
view_type = d['sfbase__cases']
source = d['sfbase__cases']['source_table']

In [31]:
bool(len(source.split('.')) < 4)

True

- look for explore and connection

In [25]:
conn = explore['conn']
database = connection_map[conn]['database']
provider = connection_map[conn]['type'].split(' ')[0]
source2 = f'{provider}.{database}.{source}'

In [26]:
source2

'Amazon.production.salesforce.cases'

In [32]:
bool(len(source2.split('.')) < 4)

False

In [ ]:
# d['sfbase__contacts']
# d['sfbase__users']
# d['sfbase__accounts']

## type 3: if 'view_type' == 'extension'
- extends

In [11]:
d['sf__contacts']

{'view_name': 'sf__contacts',
 'source_table': 'sfbase__contacts',
 'view_type': 'extension'}

- get a mapping of file names and view names

In [12]:
view_dict

{'sfbase__accounts': 'view-sf__accounts-sfbase__accounts.json',
 'sf__accounts': 'view-sf__accounts-sf__accounts.json',
 'snowflake_sf__accounts': 'view-sf__accounts-snowflake_sf__accounts.json',
 'total_active_node_count': 'view-sf__accounts-total_active_node_count.json',
 'sf__cases': 'view-sf__cases-sf__cases.json',
 'sfbase__users': 'view-sf__users-sfbase__users.json',
 'sf__contacts': 'view-sf__contacts-sf__contacts.json',
 'sfbase__cases': 'view-sf__cases-sfbase__cases.json',
 'sf__users': 'view-sf__users-sf__users.json',
 'sfbase__contacts': 'view-sf__contacts-sfbase__contacts.json'}

In [15]:
new_target_file = view_dict[d['sf__contacts']['source_table']]

In [17]:
new_target_file

'view-sf__contacts-sfbase__contacts.json'

In [21]:
with open(f'../../maps/{new_target_file}', 'r') as f:
    source_payload = json.load(f)

In [22]:
source_payload

{'view_name': 'sfbase__contacts',
 'source_table': 'salesforce.contacts',
 'view_type': 'sql_table_name'}

In [ ]:
# d['sf__users']
# d['sf__cases']
# d['sf__accounts']

## type 4
- derived table

In [ ]:
d['total_active_node_count']

## type 5
- NDT

## Bonus: chained extension

In [33]:
view1 = {"view_name": "top_contacts",
    "source_table": "middle_contacts",
    "view_type": "extension"
    }

In [37]:
view2 = {"view_name": "middle_contacts",
    "source_table": "bottom_contacts",
    "view_type": "extension"
    }

In [38]:
view3 = {"view_name": "bottom_contacts",
    "source_table": "base_contacts",
    "view_type": "extension"
    }

In [44]:
view4 = {"view_name": "snowflake_sf__accounts",
    "source_table": "SEGMENT.SALESFORCE.ACCOUNTS",
    "view_type": "sql_table_name"}

In [35]:
view_dict = {'top_contacts': 'testing_top.json',
             'middle_contacts': 'testing_middle.json',
             'bottom_contacts': 'testing_bottom.json',
             'base_contacts': 'testing_base.json'
            }

In [39]:
def is_base_view(view_payload):
    return bool(view_payload['view_type'] == 'sql_table_name')

In [45]:
view_list = [view1, view2, view3, view4]

In [46]:
i = 0

In [49]:
while(is_base_view(view_list[i])):
    print(view_list[i])
    i += 1

In [50]:
is_base_view(view_list[3])

True